In [3]:
import pandas as pd
%pip install gensim
import gensim
import gensim.corpora as gc

Le but est de réaliser une LDA avec en nombre de topics le nombre de mouvements littéraires et en corpus les textes issus des pages wikipédia. On espère que la méthode donnera des clusters correspondant aux différents mouvements littéraires (les auteurs gothiques dans un même cluster, les auteurs romantiques dans un même cluster etc.)

In [4]:
data= pd.read_csv('auteurs_complete2.csv',index_col=[0])
Mouvements = list(set(data.Mouvement.values))
topics  = len(Mouvements)  # nombre de topics 

In [5]:
mov_to_num={}
num_to_mov={}
for k in range(topics):
    mov_to_num[Mouvements[k]]=k
    num_to_mov[k]=Mouvements[k]

print('Exemple Beat Generation: ',mov_to_num['Beat Generation'])

Exemple Beat Generation:  4


In [6]:
len(set(data.Mouvement.values))

39

In [7]:
corpus  = data.sum_lem.tolist() #je prends les résumés lemmatisés (sum_lem) comme corpus.
corpus[0].split()[:10]

['William',
 'Seward',
 'Burroughs',
 'ˈsuɚd',
 'dit',
 'William',
 'Burroughs',
 'naître',
 '5',
 'février']

In [8]:
processed=[]
for aut in corpus:      # découpage des textes
    L=aut.split()
    processed.append(L)

In [9]:
dico = gc.Dictionary(processed)
print(dico)

Dictionary(8605 unique tokens: ['1914', '1997', '2', '4664', '5']...)


In [10]:
# On convertit chaque document en bag-of-words (format (token-id, nb apparition)
DTM = [dico.doc2bow(txt) for txt in processed]
DTM[0][:10]

[(0, 1),
 (1, 1),
 (2, 1),
 (3, 1),
 (4, 1),
 (5, 1),
 (6, 1),
 (7, 2),
 (8, 1),
 (9, 1)]

In [11]:
LDA = gensim.models.ldamodel.LdaModel
ldamodel = LDA(DTM,topics,id2word=dico,passes=1,random_state=0,eval_every=None) #première LDA "test" 

On voit comment sont déterminés les topics pour ce test avec la ligne suivante.<br>
On comprend aussi pourquoi les résultats seront assez désastreux : **les mots significatifs ne correspondent absolument pas à des mouvements littéraires, plutôt à l'aspect "biographique"** (naissance, mort, nationalité...)

In [12]:
ldamodel.print_topics()[:3]

[(29,
  '0.011*"il" + 0.010*"le" + 0.010*"naître" + 0.008*"plus" + 0.008*"français" + 0.007*"œuvre" + 0.007*"mort" + 0.006*"avoir" + 0.006*"comme" + 0.005*"poète"'),
 (26,
  '0.012*"il" + 0.011*"avoir" + 0.009*"le" + 0.008*"français" + 0.006*"œuvre" + 0.005*"amour" + 0.005*"poète" + 0.005*"écrivain" + 0.004*"siècle" + 0.004*"naître"'),
 (37,
  '0.013*"le" + 0.012*"mort" + 0.012*"naître" + 0.010*"écrivain" + 0.010*"avoir" + 0.009*"français" + 0.008*"plus" + 0.008*"il" + 0.007*"Paris" + 0.006*"comme"')]

In [134]:
#ldamodel.print_topics(num_topics = topics)

In [13]:
len(ldamodel[DTM]) # je vérifie que j'applique bien aux bonnes données : ok car le dataset a bien 622 entrées.

622

In [14]:
AUTEURS = []

In [15]:
c = 0
for i in ldamodel[DTM]:
    #print(data['Nom'][c],c,i)
    
    c+=1
    AUTEURS.append(i)

In [16]:
# Interprétation
print("La LDA prédit pour", data['Nom'][0], ' le topic', AUTEURS[0][0][0], ' avec une proba ', AUTEURS[0][0][1])

La LDA prédit pour William S. Burroughs  le topic 2  avec une proba  0.98500586


In [17]:
#On veut retourner pour un auteur donné le topic avec la proba la plus élevée 
def tri(L):
    cand = 0
    for k in range(len(L)):
        (a,b)=L[k]
        if b>L[cand][1]:
            cand=k
    return L[cand]

In [18]:
data['LDA']=AUTEURS

In [19]:
data['LDA']=data['LDA'].apply(tri)

In [20]:
data['LDA']=data['LDA'].apply(lambda c: c[0])

In [21]:
data[data['LDA']==3] # on regarde quels auteurs ont été assignés au topic 3. On voit qu'ils sont issus de mouvements très différents...

,Nom,Mouvement,summary,sum_lem,mvm_label,page,page_lem,is_empty,LDA
181,Gérard de Nerval,romantique,"Gérard Labrunie, dit Gérard de Nerval, est un ...",Gérard Labrunie dire Gérard Nerval écrivain po...,34,Gérard de Nerval (French: [ʒeʁaʁ də nɛʁval]; 2...,Gérard nerval French ʒeʁaʁ də nɛʁval 22 may 18...,1,3
208,Wilhelm Ténint,romantique,"Wilhelm Ténint est un écrivain français, né à ...",Wilhelm Ténint écrivain français naître Paris ...,34,"Wilhelm Ténint (20 May 1817, Paris – 26 April ...",Wilhelm Ténint 20 may 1817 Paris 26 april 1879...,1,3
274,Guilhem_Figueira,troubadour,Guillem ou Guilhem Figueira ou Figera (Toulous...,guillem Guilhem Figueira Figera Toulouse 1195 ...,38,Guillem or Guilhem Figueira or Figera was a La...,Guillem or Guilhem figueira or Figera wa avoir...,1,3
323,Rémy_Belleau,pleiade,"Rémy Belleau, né à Nogent-le-Rotrou en 1528, m...",Rémy Belleau naître 1528 mort Paris 6 mars 157...,27,Remy (or Rémi) Belleau (1528 – 6 March 1577) w...,Remy or Rémi Belleau 1528 6 March 1577 was avo...,1,3
337,Pierre_Carlet_de_Chamblain_de_Marivaux,picaresque,Marivaux (né Pierre Carlet) est un écrivain fr...,marivaux naître Pierre Carlet écrivain françai...,26,Pierre Carlet de Chamblain de Marivaux (4 Febr...,Pierre Carlet Chamblain marivaux 4 february 16...,1,3
427,Pierre_Carlet_de_Chamblain_de_Marivaux,Lumieres,Marivaux (né Pierre Carlet) est un écrivain fr...,marivaux naître Pierre Carlet écrivain françai...,6,Pierre Carlet de Chamblain de Marivaux (4 Febr...,Pierre Carlet Chamblain marivaux 4 february 16...,1,3
538,Constant_Malva,proletarienne,Constant Malva de son vrai nom Alphonse Bourla...,conster Malva vrai nom alphonse Bourlard naîtr...,30,"Malva preissiana, the Australian hollyhock or...",Malva preissiana the Australian hollyhock or n...,1,3
580,Alain_Borne,Ecole de Rochefort,"Alain Borne, né le 12 janvier 1915 à Saint-Pon...",Alain Borne naître 12 janvier 1915 Allier mour...,2,Alain Borne (12 January 1915 – 21 December 196...,Alain Borne 12 January 1915 21 december 1962 w...,1,3
585,Louis_Parrot,Ecole de Rochefort,"Louis Parrot, né à Tours le 28 août 1906 et mo...",Louis parrot naître tour 28 août 1906 mort Par...,2,Louis Parrot (28 August 1906 – 24 August 1948)...,Louis Parrot 28 august 1906 24 august 1948 was...,1,3


In [22]:
def Lda_(number_topics,corpus):
    processed=[]
    for aut in corpus:
        L=aut.split()
        processed.append(L)
    dico = gc.Dictionary(processed)
    DTM = [dico.doc2bow(txt) for txt in processed]
    LDA = gensim.models.ldamodel.LdaModel
    ldamodel = LDA(DTM,topics,id2word=dico,passes=1,random_state=0,eval_every=None)
    return ldamodel,DTM
    
    #formalisatio ndans une fonction

In [23]:
id2word = gc.Dictionary()

In [24]:
corpus = data.page_lem.tolist() # on prend cette fois ci en corpus la page entière

In [25]:
LDA  = Lda_(39,corpus) #on demande de créer 39 topics

In [26]:
def results_lda(LDA,df_,DTM):
    AUTEURS=[]
    c = 0
    for i in LDA[DTM]:
        AUTEURS.append(i)
    result = df_[['Nom','Mouvement']]
    result['cluster']=AUTEURS
    result['cluster']=result['cluster'].apply(tri)
    result['cluster']=result['cluster'].apply(lambda a:a[0])
    return result

In [44]:
for mov in set(data.Mouvement.values):
    
    print(mov,str(len(data[data['Mouvement']==mov])))
    # j'affiche la taille de chaque mouvement afin d'afficher le resultat d'une LDA réduite sur un nombre limité de mouvements (mais avec assez d'auteurs) afin de pouvoir interpréter.

Hussards 16
Négritude 9
picaresque 21
décadentisme 28
Beat Generation 11
absurde 7
Renaixença 25
rhetoriqueurs 14
surréaliste 41
preromantisme 13
gothique 28
Ecole de Rochefort 37
Naturalisme 26
pleiade 8
quietisme 5
fabliau 3
objectivisme 6
Ecole romane 5
preciosite 1
Nouveau Roman 7
Parnasse 10
existentialisme 8
Realisme magique 9
classique 5
du courant réaliste 39
symboliste 5
humanisme 20
Lumieres 31
régionaliste 17
oulipo 9
proletarienne 21
hagiographie 4
roman courtois 1
Courant de conscience 4
romantique 91
troubadour 16
baroque 5
jungien 5
Génération perdue 11


In [45]:
# sous LDA
courants = ['Nouveau Roman','Realisme magique','Négritude','gothique','décadentisme','Naturalisme']
data['sous_df'] = data['Mouvement'].apply(lambda s : int(s in courants))
sous_courants = data[data['sous_df']==1][['Nom','Mouvement','sum_lem','page_lem']]

In [46]:
txt = sous_courants.page_lem.tolist() # je récupère les pages wiki en corpus

In [47]:
LDA,DTM = Lda_(6,txt)

In [48]:
resultats = results_lda(LDA,sous_courants,DTM) 

<ipython-input-26-cc163adb9326>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['cluster']=AUTEURS
<ipython-input-26-cc163adb9326>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result['cluster']=result['cluster'].apply(tri)
<ipython-input-26-cc163adb9326>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide

In [49]:
resultats[resultats['Mouvement']=='gothique']

#Je regarde à quel cluster ont été associés les auteurs gothiques.
# Beaucoup sont associés au cluster 30 mais le reste est assez dispersé...

,Nom,Mouvement,cluster
364,Claudine_Guérin_de_Tencin,gothique,30
365,Horace_Walpole,gothique,30
366,John_Aikin,gothique,13
367,Clara_Reeve,gothique,30
369,William_Thomas_Beckford,gothique,30
371,Mary_Ann_Radcliffe,gothique,32
372,Eliza_Parsons,gothique,36
373,Ann_Radcliffe,gothique,30
375,Regina_Maria_Roche,gothique,36
377,Charles_Robert_Maturin,gothique,30


In [50]:
accuracy_ish={}
for courant in courants:
    L=list(resultats[resultats['Mouvement']==courant].cluster.values)
    n=len(set(L))
    l=len(L)
    cour={}
    for k in range(l):
        if L[k] in cour:
            cour[L[k]]+=1
        else:
            cour[L[k]]=1
    accuracy_ish[courant]=(n,l,cour)
    

# accuracy_ish va nous expliciter les résultats de répartition des auteurs par mouvement dans différents clusters.

In [51]:
accuracy_ish

# premier chiffre : n, nombre de clusters différents dans lesquels la LDA a associé les auteurs
# deuxième : l, le nombre d'auteurs dans le mouvement
# dictionnaire : quels clusters trouve-t-on pour les auteurs de ce mouvement
#                et combien d'auteurs trouve-t-on par cluster

{'Nouveau Roman': (5, 7, {4: 1, 36: 1, 13: 3, 27: 1, 37: 1}),
 'Realisme magique': (5, 9, {36: 2, 13: 3, 30: 2, 33: 1, 32: 1}),
 'Négritude': (4, 9, {36: 2, 30: 5, 13: 1, 0: 1}),
 'gothique': (8, 28, {30: 17, 13: 1, 32: 1, 36: 3, 7: 1, 33: 3, 22: 1, 0: 1}),
 'décadentisme': (11,
  28,
  {30: 9, 13: 7, 22: 2, 37: 2, 34: 1, 2: 2, 32: 1, 36: 1, 35: 1, 0: 1, 27: 1}),
 'Naturalisme': (8,
  26,
  {30: 12, 36: 5, 13: 4, 33: 1, 1: 1, 32: 1, 27: 1, 35: 1})}

In [55]:
# LDA.print_topics() montre encore des topics peu significatifs...

### Lecture des données : 

Les auteurs du Nouveau Roman ont été placés dans 5 clusters différents alors qu'ils ne sont que 5. On peut penser que la clusterisation est donc assez mauvaise, au sens où on ne semble pas pouvoir retrouver chaque mouvement littéraire dans un cluster.
Par exemple : la dispersion est moindre pour les auteurs gothiques. (28 auteurs, 8 clusters). Pour autant, le cluster au sein duquel ils ont majoritairement été placés (le numéro 30) est également le cluster principal pour le Naturalisme. La LDA, pour l'instant, ne permet pas de retrouver les mouvements littéraires.

On peut imputer ça à plusieurs choses : peut-être que les clusters ne sont pas si bien définis (les mouvements littéraires n'ont pas forcément de définition fixe exprimable simplement, ce sont des notions complexes, et rien ne dit que les pages wikipédia rendent bien les distinctions).
Il faudrait par ailleurs plus de temps pour mieux appréhender le type d'erreurs : associer des auteurs et autrices réalistes avec des naturalistes n'est pas aussi "choquant" que d'associer le réalisme magique et les fabliaux. Cela demanderait cependant beaucoup de temps et une expertise sur l'ensemble des mouvements scrappés que nous n'avons pas nécessairement ! 
